In [ ]:
from ai_toolkits.structured.extractor import create_object_openai
from pydantic import BaseModel, Field
from ai_toolkits.llms import create_sync_client

class Person(BaseModel):
    name: str = Field(..., description="The person's full name")
    age: int = Field(..., description="The person's age in years")
    email: str = Field(..., description="The person's email address")

obj = create_object_openai(
    output_cls=Person, 
    prompt="I saw a girl on the street, I asked her name, she said her name is Tendy Alice, she is 30 years old, her email is alice@example.com", 
    client=create_sync_client()
)

In [31]:
obj

Person(name='Tendy Alice', age=30, email='alice@example.com')

In [5]:
import asyncio
from llama_index.core.agent.workflow import FunctionAgent
from ai_toolkits.llms import LlamaIndeAzureOpenAI

import nest_asyncio
nest_asyncio.apply()

# Define a simple calculator tool
def multiply(a: float, b: float) -> float:
    """Useful for multiplying two numbers."""
    print(f"Multiplying {a} and {b}")
    return a * b


# Create an agent workflow with our calculator tool
agent = FunctionAgent(
    tools=[multiply],
    llm=LlamaIndeAzureOpenAI(engine="gpt-4o", model="gpt-4o", temperature=0),
    system_prompt="You are a helpful assistant that can multiply two numbers.",
)


async def main():
    # Run the agent
    response = await agent.run("What is 1234 * 4567?, Be sure to use tools when doing math.")
    print(str(response))


# Run the agent
if __name__ == "__main__":
    asyncio.run(main())

Multiplying 1234 and 4567
The result of \(1234 \times 4567\) is \(5635678\).


In [7]:
from ai_toolkits.llms import create_async_client

client = create_async_client()

In [12]:
from openai import AsyncOpenAI

from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.openai import OpenAIProvider
model = OpenAIChatModel(
    'gpt-4o', 
    provider=OpenAIProvider(openai_client=client)
)
agent = Agent(model)

In [9]:
agent.run_sync("Write a poem about the sea.")

AgentRunResult(output="**The Eternal Sea**\n\nOh, endless sea, a boundless gray,  \nYou shimmer soft at break of day.  \nUpon your breath, the gulls alight,  \nWhile waves cascade in foamy flight.  \n\nYou cradle secrets in your depths,  \nOld whispered tales from each salt breath.  \nOf ships long lost, of hearts once free,  \nNow part of your eternity.  \n\nBeneath your surface, life abounds,  \nA hidden realm where silence sounds.  \nWith darting fish and coral's hue,  \nThe ocean dreams in shades of blue.  \n\nYour tides may dance, your storms may roar,  \nYet still you kiss the aching shore.  \nA timeless rhythm, fierce and bold,  \nThat soothes the heart and stirs the soul.  \n\nOh, mighty sea, both wild and vast,  \nYou link the present to the past.  \nAn endless muse, a mystery—  \nForever shall I love the sea.  ")

In [13]:
import os

import logfire
from pydantic import BaseModel

from pydantic_ai import Agent

# 'if-token-present' means nothing will be sent (and the example will work) if you don't have logfire configured
logfire.configure(send_to_logfire='if-token-present')
logfire.instrument_pydantic_ai()


class MyModel(BaseModel):
    city: str
    country: str

agent = Agent(model, output_type=MyModel)

if __name__ == '__main__':
    result = agent.run_sync('The windy city in the US of A.')
    print(result.output)
    print(result.usage())

13:45:44.889 agent run
13:45:44.890   chat gpt-4o
city='Chicago' country='USA'
RunUsage(input_tokens=58, output_tokens=19, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, requests=1)


In [55]:
from pydantic import BaseModel, Field
import uuid
from typing import Optional

class DirectedEdge(BaseModel):
    """An edge in a graph, if directed, goes from source_node to target_node, else it is undirected.
    edge type defines a relationship between two nodes.
    edge info is a dictionary that can contain any additional information about the edge.
    """
    edge_id:Optional[str] = Field(
        default_factory=lambda: str(uuid.uuid4())[:6], 
        description="The unique identifier of the edge, usually auto-generated")
    source_node_id: str = Field(..., description="The ID of the source node")
    target_node_id: str = Field(..., description="The ID of the target node")   
    edge_type:str = Field(None, description="The type of the edge, e.g., 'friendship', 'transaction', etc.")
    edge_attr:dict = Field(default_factory=dict, description="A dictionary of attributes for the edge")
    directed:bool = Field(default=True, description="Whether the edge is directed or not")

class Node(BaseModel):
    """Represents a node in a graph."""
    
    node_id:Optional[str] = Field(
        default_factory=lambda: str(uuid.uuid4())[:6], 
        description="The unique identifier of the node, usually auto-generated")
    node_name:str = Field(..., description="The name of the node")
    node_type:str = Field(..., description="The type of the node")
    node_attr:dict = Field(default_factory=dict, description="A dictionary of attributes for the node")
    
    
class Graph(BaseModel):
    """A simple graph structure with nodes and edges."""
    
    nodes:list[Node] = Field(default_factory=list, description="A list of nodes in the graph")
    edges:list[DirectedEdge] = Field(default_factory=list, description="A list of edges in the graph")
    graph_desc:Optional[str] = Field(None, description="A description of the graph")   
    
    def add_node(self, node:Node):
        self.nodes.append(node)
        
    def add_edge(self, edge:DirectedEdge):
        self.edges.append(edge) 
        
    def one_step_upstream_nodes(self, node_id:str) -> list[Node]:
        """Get all upstream nodes of a given node."""
        upstream_node_ids = {edge.source_node_id 
                             for edge in self.edges 
                             if edge.target_node_id == node_id and edge.directed}
        return [node for node in self.nodes if node.node_id in upstream_node_ids]
    
    def one_step_downstream_nodes(self, node_id:str) -> list[Node]:
        """Get all downstream nodes of a given node."""
        downstream_node_ids = {edge.target_node_id 
                               for edge in self.edges 
                               if edge.source_node_id == node_id and edge.directed}
        return [node for node in self.nodes if node.node_id in downstream_node_ids]

graph = Graph(graph_desc="A simple social graph")

def add_node(
    node_name:str, 
    node_type:str, 
    node_attr:dict=None):
    """Add a node to the graph."""
    graph.add_node(
        node_name=node_name, 
        node_type=node_type, 
        node_attr=node_attr or {})
    
def add_edge(
    source_node_id:str, 
    target_node_id:str, 
    edge_type:str, 
    edge_attr:dict=None, 
    directed:bool=True):
    """Add an edge to the graph."""
    graph.add_edge(
        source_node_id=source_node_id, 
        target_node_id=target_node_id, 
        edge_type=edge_type, 
        edge_attr=edge_attr or {}, 
        directed=directed
    )
   
node1 = Node(node_name="Alice", node_type="Person", node_attr={"age": 30, "city": "New York"})
node2 = Node(node_name="Bob", node_type="Person", node_attr={"age": 25, "city": "San Francisco"})
edge = DirectedEdge(
    source_node_id=node1.node_id, 
    target_node_id=node2.node_id, 
    edge_type="friendship", 
    edge_attr={"since": 2020})

graph.add_node(node1)
graph.add_node(node2)
graph.add_edge(edge)